In [1]:
 !pip install selenium
 !pip install beautifulsoup4
 !pip install requests
 !pip install lxml

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Для того, что бы все работало, нужно скачать chromedriver

Ссылка откуда можно скачать chromedriver https://chromedriver.chromium.org/downloads.

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from time import sleep
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np

Проверочный скрипт, который выдает количество фильмов на первой странице. Если все установлено верно, ты выдаст в ответе 50

In [3]:
browser = Chrome('chromedriver')
url = f'https://ru.kinorium.com/collections/critics/131/?order=sequence&page=1&perpage=50&show_viewed=1'
browser.get(url)
browser.page_source
soup = BeautifulSoup(browser.page_source, 'lxml')
films = soup.findAll('div', class_='filmList__item-content')
print(len(films))

50


In [4]:
def film_control(url):
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    budget = soup.find('div', class_='film-page__infowrap').find('span', class_='box-budget-tooltip')
    fees = soup.find('div', class_='film-page__infowrap').find('span', class_='film-page__gross-switcher-control')
    return(budget, fees)

In [5]:
df = pd.DataFrame({'Русское название':[], 'Оригинальное название':[], 'Год':[], 'Рейтинг IMDb':[], 'Страна':[], 'Режиссер':[], 'Бюджет ($)':[], 'Cборы в Мире ($)':[], 'Ссылка':[]})
browser = Chrome('chromedriver')
# df = pd.DataFrame({'Русское название':[], 'Оригинальное название':[], 'Год':[], 'Рейтинг IMDb':[], 'Страна':[], 'Режиссер':[], 'Ссылка':[]})

for i in range(1,21):
    print(f'Страница {i}')
    url = f'https://ru.kinorium.com/collections/critics/131/?order=sequence&page={i}&perpage=50&show_viewed=1'
    browser.get(url)
    sleep(3)  
    soup = BeautifulSoup(browser.page_source, 'lxml')
    films = soup.findAll('div', class_='filmList__item-content')
    for film in films:
        link = 'https://ru.kinorium.com'+ film.find('a', class_='filmList__item-title item__title filmList__item-title-link').get('href')
        rus_name = film.find('a', class_='filmList__item-title item__title filmList__item-title-link').find('i', class_='movie-title__text filmList__item-title-link-popup link-info-movie-type-film').text.strip()
        *eng_name, year = film.find('div', class_='info-box filmList__item-title-wrap').find('div', class_='info').find('div', class_='select-wrap').text.strip().split(', ')
        IMDb_rating = film.find('div', class_='info-box filmList__item-title-wrap').find('div', class_='info').find('div', class_='item__link-rating').find('li', class_='rating_imdb').text.strip().split(' :  ')[1]
        country, director = film.find('div', class_='info-box filmList__item-title-wrap').find('div', class_='info').find('p', class_='filmList__extra-info').find('span', class_='filmList__extra-info-director').text.strip().split(' •  ')
        budget, fees = film_control(link)
        if budget != None:
            budget = budget.text.strip()
            if "CA$" in budget:
                budget = budget.replace('CA$', '')
                budget = int(''.join(budget.split()))
            elif "A$" in budget:
                budget = budget.replace('A$', '')
                budget = int(''.join(budget.split()))
            elif "$" in budget:
                budget = budget.replace('$', '')
                budget = int(''.join(budget.split()))
            else:
                budget = None

        if fees != None:
            fees = fees.text.strip().replace('$', '').replace('Р', '')
            fees = int(''.join(fees.split()))
        if "Топ-500" in rus_name:
            rus_name = rus_name.replace('Топ-500', '')
        if '...' in director:
            director = director.replace('...', '')
        if eng_name == []:
            eng_name = [rus_name]

        df.loc[len(df.index)] = [rus_name, ''.join(eng_name), year, IMDb_rating, country, director, budget, fees, link]

print('Готово!')

Страница 1
Страница 2
Страница 3
Страница 4
Страница 5
Страница 6
Страница 7
Страница 8
Страница 9
Страница 10
Страница 11
Страница 12
Страница 13
Страница 14
Страница 15
Страница 16
Страница 17
Страница 18
Страница 19
Страница 20
Готово!


In [6]:
df['Рейтинг IMDb'] = df['Рейтинг IMDb'].astype('float')

In [7]:
df = df.fillna(df.mean()).round(0)

C:\Users\Дарья\AppData\Local\Temp\ipykernel_13584\706233491.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df = df.fillna(df.mean()).round(0)


Заполняем пропуски в данных средними значениями. В основном пропуски в колонках бюджет и сборы по миру, на этом сайте нет информации о бюджете части фильмов, и среднее знаечние для бюджета будет оптимальным, так как не смещает выборку. Возможно для каких-то фильмов в действительности средние значения бюджета далеки от реальности, но такая жизнь, nobody gonna know.

Сохранение базы данных фильмов в формат csv

In [8]:
df.to_csv('films_result.csv', index=False, sep=';', encoding='utf-8-sig')

In [9]:
df.head(20)

,Русское название,Оригинальное название,Год,Рейтинг IMDb,Страна,Режиссер,Бюджет ($),Cборы в Мире ($),Ссылка
0,Туз в рукаве,Ace in the Hole,1951,8.0,США,Билли Уайлдер,1800000.0,2300000.0,https://ru.kinorium.com/41033/
1,Эйс Вентура: Розыск домашних животных,Ace Ventura: Pet Detective,1993,7.0,США,Том Шэдьяк,15000000.0,107217396.0,https://ru.kinorium.com/99229/
2,Месть актера,Yukinojô henge,1963,7.0,Япония,Кон Итикава,11398150.0,49982288.0,https://ru.kinorium.com/54246/
3,Зависимость,The Addiction,1994,6.0,США,Абель Феррара,11398150.0,307308.0,https://ru.kinorium.com/102068/
4,Страховой агент,The Adjuster,1991,7.0,Канада,Атом Эгоян,1500000.0,396573.0,https://ru.kinorium.com/92500/
5,Приключения Бакару Банзая в восьмом измерении,The Adventures of Buckaroo Banzai Across the 8...,1984,6.0,США,У. Д. Рихтер,11398150.0,6254148.0,https://ru.kinorium.com/79880/
6,Приключения Робин Гуда,The Adventures of Robin Hood,1938,8.0,США,Майкл Кёртиц,1900000.0,3981000.0,https://ru.kinorium.com/28192/
7,После работы,After Hours,1985,8.0,США,Мартин Скорсезе,4500000.0,10609321.0,https://ru.kinorium.com/81475/
8,"Агирре, гнев божий",Aguirreder Zorn Gottes,1972,8.0,"Германия (ФРГ), Мексика",Вернер Херцог,370000.0,37041.0,https://ru.kinorium.com/63490/
9,Аэроплан,Airplane!,1980,8.0,США,Джим Абрахамс,3500000.0,83453539.0,https://ru.kinorium.com/74115/
